In [1]:
import os
os.chdir('../')
%pwd

'd:\\A - My Projects\\A - MLOps\\Flower-Gift-Helper'

# Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    data_dir: Path
    updated_model_path: Path
    trained_model_path: Path
    params_validation_split: float
    params_input_shape: list
    params_batch_size: int
    params_is_early_stopping: bool
    params_epochs: int
    params_learning_rate: float

# Configuration Manager

In [3]:
from flowerClassifier import logger, CONFIG_FILE_PATH, PARAMS_FILE_PATH
from flowerClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigManager():
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root_dir])
        
    def get_model_training_config(self) -> ModelTrainingConfig:
        
        create_directories([self.config.prepare_base_model.folder_dir])
        
        model_training_config = ModelTrainingConfig(
                data_dir=self.config.data_ingestion.data_dir,
                updated_model_path=self.config.prepare_base_model.updated_model_path,
                trained_model_path=self.config.model_training.trained_model_path,
                params_validation_split=self.params.VALIDATION_SPLIT,
                params_input_shape=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE,
                params_is_early_stopping=self.params.EARLY_STOPPING,
                params_epochs=self.params.EPOCHS,
                params_learning_rate=self.params.LEARNING_RATE
        )
        
        return model_training_config

# Component

In [8]:
from pathlib import Path
import tensorflow as tf
from flowerClassifier.utils.common import save_json

In [11]:

class TrainBaseModel():
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def save_mapping(self):
        save_json(path=Path("constant/class_mapping.json"), data=self.train_generator.class_indices)
    
    def train_valid_generator(self):
        image_data_generator_kwargs=dict(
            rescale=1./255,
            validation_split=self.config.params_validation_split
            )
            
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            **image_data_generator_kwargs
        )
        
        validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **image_data_generator_kwargs
        )
        
        # configure data flow
        data_flow_kwargs = dict(
            target_size=self.config.params_input_shape[:2],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            interpolation="bilinear"
        )
                
        self.train_generator = train_datagen.flow_from_directory(
            directory = self.config.data_dir,
            subset='training',
            shuffle=True,
            ** data_flow_kwargs
        )
        
        self.save_mapping()
        
        self.validation_generator = validation_datagen.flow_from_directory(
            directory = self.config.data_dir,
            subset='validation',
            shuffle=False,
            ** data_flow_kwargs
        )
        
    def train_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_model_path)

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

        if self.config.params_is_early_stopping:
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                              patience=5, 
                                                              restore_best_weights=True)
                
            self.model.fit(self.train_generator,
                           validation_data=self.validation_generator,
                           epochs=self.config.params_epochs,
                           callbacks=[early_stopping])
        else:
            self.model.fit(self.train_generator, 
                           validation_data=self.validation_generator,
                           epochs=self.config.params_epochs)
            
        self.save_model(self.config.trained_model_path, self.model)
        logger.info(f"Trained model successfully saved to folder {self.config.trained_model_path}")

# Pipeline

In [12]:
config = ConfigManager()
model_training_config = config.get_model_training_config()
train_base_model = TrainBaseModel(config=model_training_config)
train_base_model.train_valid_generator()
train_base_model.train_model()

[2024-09-09 16:05:47,484 | INFO] common: yaml file: constant\config.yaml loaded successfully
[2024-09-09 16:05:47,488 | INFO] common: yaml file: params.yaml loaded successfully
[2024-09-09 16:05:47,489 | INFO] common: created directory at: artifacts
[2024-09-09 16:05:47,491 | INFO] common: created directory at: artifacts/model_folder
Found 2198 images belonging to 5 classes.
[2024-09-09 16:05:47,585 | INFO] common: json file saved at: constant\class_mapping.json
Found 548 images belonging to 5 classes.
